In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

# Choose a name for your CPU cluster

cpu_cluster_name = "bm-lr-ml"

compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)

compute_target = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform
import os

# Specify parameter sampler
ps = RandomParameterSampling( {
    ### YOUR CODE HERE ###
    "C": uniform(0.1, 1) #Explorar valores de C en el espacio 0.1 a 1 
    "max_iter": choice (100, 150, 200,  250) #Escoger iteraciones en el espacio 100 a 250
    }
)    

# Specify a Policy
### YOUR CODE HERE ###
policy = BanditPolicy(slack_factor = 0.1,
                     evaluation_interval = 1,
                     delay_evaluation = 5) 

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
### YOUR CODE HERE ###
est = SKLearn(
            source_directory = ".",
            compute_target = compute_target,
            entry_script = "train.py"
)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
### YOUR CODE HERE ###
hyperdrive_config = Hyperdrive_Config(hyperparameter_sampling = ps,
                                       primary_metric_goal = PrimaryMetricGoal.MAXIMIZE,
                                       primary_metric_name = "Accuracy",
                                       max_concurrent_runs = 4,
                                       max_total_runs = 40,
                                       policy = policy,
                                       estimator = est)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

### YOUR CODE HERE ###
hyperdrive_run <- exp.submit(hyperdrive_config, show_output = True)

RunDetails(hyperdrive_run).show

In [ ]:
import joblib
# Get your best run and save the model from that run.

### YOUR CODE HERE ###

best_run = hyperdrive_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

print('Best Run Id: ', best_run.id)
print('\n Accuracy:', best_run_metrics['accuracy'])
print('\n learning rate:',parameter_values[3])
print('\n keep probability:',parameter_values[5])
print('\n batch size:',parameter_values[7])

model_hd = best_run.register_model(model_name='sklearn_hd', model_path='outputs/model.joblib')

os.makedirs('outputs', exist_ok=True)
joblib.dump(model_hd, 'outputs/model.joblib')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = Dataset.Tabular.from_delimited_files(path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv", separator=',', encoding='utf8')


In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.

x, y = clean_data(ds)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=37) 

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes = 30,
    task = 'classification',
    primary_metric = 'Accuracy',
    training_data = x_train,
    label_column_name = 'y',
    n_cross_validations = 5)


In [2]:
# Submit your automl run

remote_run = experiment.submit(automl_config, show_output = False)

RunDetails(remote_run).show

In [1]:
# Retrieve and save your best automl model.

### YOUR CODE HERE ###

best_run, fitted_model = remote_run.get_output()
print(best_run)
print(fitted_model)

model_aml = best_run.register_model(model_name='automl', model_path='outputs/model.joblib')

joblib.dump(model_aml, 'outputs/model.joblib')

NameError: name 'remote_run' is not defined